<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


 - Install the additional package requirements for this bonus notebook by uncommenting and running the following cell:
 - 通过取消注释并运行以下单元格来安装此附加笔记本的额外包依赖:

In [14]:
 #pip install -r requirements-extra.txt

 # Comparing Various Byte Pair Encoding (BPE) Implementations
 # 比较不同的字节对编码(BPE)实现

<br>
&nbsp;

 ## Using BPE from `tiktoken`
 ## 使用`tiktoken`的BPE实现

In [15]:
# 导入importlib.metadata模块以获取包版本信息
from importlib.metadata import version

# 打印tiktoken包的版本号
print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.8.0


In [16]:
# 导入tiktoken库用于分词
import tiktoken

# 获取GPT-2的分词器
tik_tokenizer = tiktoken.get_encoding("gpt2")

# 定义要分词的示例文本
text = "Hello, world. Is this-- a test?"

In [17]:
# 使用tiktoken分词器将文本编码为整数token序列
# allowed_special参数指定允许的特殊token
integers = tik_tokenizer.encode(text, allowed_special={"<|endoftext|>"})

# 打印编码后的整数token序列
print(integers)

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]


In [18]:
# 使用tiktoken分词器将整数token序列解码回文本
strings = tik_tokenizer.decode(integers)

# 打印解码后的文本
print(strings)

Hello, world. Is this-- a test?


In [19]:
# 打印tiktoken分词器的词汇表大小
print(tik_tokenizer.n_vocab)

50257


<br>
&nbsp;

 ## Using the original BPE implementation used in GPT-2
## 使用GPT-2中的原始BPE实现

In [20]:
# 从bpe_openai_gpt2模块导入get_encoder和download_vocab函数
# get_encoder用于获取GPT-2的BPE编码器
# download_vocab用于下载GPT-2的词汇表文件
from bpe_openai_gpt2 import get_encoder, download_vocab

In [21]:
# 下载GPT-2模型所需的词汇表文件
# 包括encoder.json(token到id的映射)和vocab.bpe(BPE合并规则)
download_vocab()

Fetching encoder.json: 1.04Mit [00:01, 664kit/s]                                                    
Fetching vocab.bpe: 457kit [00:00, 500kit/s]                                                        


In [22]:
# 使用get_encoder函数获取GPT-2的BPE分词器
# model_name指定模型名称为"gpt2_model"
# models_dir指定模型文件所在目录为当前目录"."
orig_tokenizer = get_encoder(model_name="gpt2_model", models_dir=".")

In [23]:
# 使用GPT-2原始BPE分词器对文本进行编码
# encode()方法将文本转换为整数token序列
integers = orig_tokenizer.encode(text)

# 打印编码后的整数token序列
print(integers)

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]


In [24]:
# 使用GPT-2原始BPE分词器对整数token序列进行解码
# decode()方法将整数token序列转换回原始文本
strings = orig_tokenizer.decode(integers)

# 打印解码后的文本
print(strings)

Hello, world. Is this-- a test?


<br>
&nbsp;

 ## Using the BPE via Hugging Face transformers
 ## 通过 Hugging Face transformers 使用 BPE

In [25]:
# 导入Hugging Face的transformers库
import transformers

# 打印transformers库的版本号
transformers.__version__

'4.46.1'

In [1]:
# 从transformers库导入GPT2Tokenizer类
from transformers import GPT2Tokenizer

# 使用from_pretrained方法加载预训练的GPT2分词器
# 参数"gpt2"指定使用GPT2预训练模型的分词器
hf_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

D:\Anaconda\Anaconda\envs\LLMs\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [ ]:
# 使用Hugging Face的GPT2分词器对文本进行编码
# 返回字典中的input_ids字段,即token的整数序列
hf_tokenizer(strings)["input_ids"]

<br>
&nbsp;

 
 ## 性能比较
 ## Performance Comparison


In [15]:
# 打开并读取文本文件
# 使用相对路径打开the-verdict.txt文件
# 指定编码为utf-8以正确处理Unicode字符
with open('../01_main-chapter-code/the-verdict.txt', 'r', encoding='utf-8') as f:
    # 读取整个文件内容到raw_text变量中
    raw_text = f.read()

In [ ]:
# 使用%timeit魔法命令测量原始分词器对文本进行编码所需的时间
# 调用orig_tokenizer的encode方法对raw_text进行编码
%timeit orig_tokenizer.encode(raw_text)

In [3]:
# 使用%timeit魔法命令测量tiktoken分词器对文本进行编码所需的时间
# 调用tik_tokenizer的encode方法对raw_text进行编码
%timeit tik_tokenizer.encode(raw_text)

NameError: name 'tik_tokenizer' is not defined

In [ ]:
# 使用%timeit魔法命令测量Hugging Face分词器对文本进行编码所需的时间
# 调用hf_tokenizer对raw_text进行编码,并获取input_ids字段
%timeit hf_tokenizer(raw_text)["input_ids"]

In [2]:
# 使用%timeit魔法命令测量Hugging Face分词器对文本进行编码所需的时间
# 设置最大长度为5145并启用截断,以避免序列过长错误
# 调用hf_tokenizer对raw_text进行编码,并获取input_ids字段
%timeit hf_tokenizer(raw_text, max_length=5145, truncation=True)["input_ids"]

NameError: name 'hf_tokenizer' is not defined